In [1]:
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings("ignore")
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

#### step1. load the model cards

In [2]:
from src.io import load_cards
data_root = "./data/"

model_cards = load_cards(data_root, "model")
print(f"There are {len(model_cards)} model repo cards")

Loading the model cards from ./data/model_cards.json
There are 200 model repo cards


In [3]:
from src.utils import show_cards

show_cards(model_cards, type="model", num=2, sample="head")


========================= The model card for the repo address: https://huggingface.co/microsoft/phi-2 =========================

Model Summary
Phi-2 is a Transformer with 2.7 billion parameters. It was trained using the same data sources as Phi-1.5, augmented with a new data source that consists of various NLP synthetic texts and filtered websites (for safety and educational value). When assessed against benchmarks testing common sense, language understanding, and logical reasoning, Phi-2 showcased a nearly state-of-the-art performance among models with less than 13 billion parameters.
Our model hasn't been fine-tuned through reinforcement learning from human feedback. The intention behind crafting this open-source model is to provide the research community with a non-restricted small model to explore vital safety challenges, such as reducing toxicity, understanding societal biases, enhancing controllability, and more.
Intended Uses
Given the nature of the training data, the Phi-2 mod

#### step2. chat with the model cards

In [18]:
from src.chat import chat_cards

each_query = "Is this model fine-tuned for some specific downstream tasks, \
like classificiation, summarization, translation, code generation, math-problem solving?\
REMEMBER: the response should start with 'YES' or 'NO', and then follow a very brief explanation"

response = chat_cards(
    query=each_query,
    cards=model_cards,
    type="model",
    llm="gpt-3.5-turbo-1106",
    mode="each",
    verbose=True,
)


========================= Chatting with the model from its card with the repo: microsoft/phi-2 =========================

NO, this model has not been fine-tuned for any specific downstream tasks. It is intended for QA, chat, and code purposes, but it has not undergone fine-tuning for specific tasks like classification, summarization, translation, or math-problem solving.

========================= Chatting with the model from its card with the repo: TinyLlama/TinyLlama-1.1B-Chat-v1.0 =========================

NO, this model is fine-tuned for chat generation and not for specific downstream tasks like classification, summarization, translation, code generation, or math-problem solving.

========================= Chatting with the model from its card with the repo: mistralai/Mixtral-8x7B-Instruct-v0.1 =========================

NO. The Mixtral-8x7B Instruct model is a quick demonstration that the base model can be easily fine-tuned to achieve compelling performance, but it does not spec

In [21]:
yes_models = []

for resp in response[0]:
    if resp['response'].startswith('YES'):
        yes_models.append(resp['repo_addr'])
        
print(f"There are {len(yes_models)} / {len(model_cards)} models that are fine-tuned for some specific downstream tasks:")
yes_models

There are 59 / 200 models that are fine-tuned for some specific downstream tasks:


['https://huggingface.co/TencentARC/LLaMA-Pro-8B',
 'https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0',
 'https://huggingface.co/CultriX/MistralTrix-v1',
 'https://huggingface.co/mlabonne/Beyonder-4x7B-v2',
 'https://huggingface.co/moreh/MoMo-70B-LoRA-V1.4',
 'https://huggingface.co/Pclanglais/Mickey-1928',
 'https://huggingface.co/NousResearch/Nous-Hermes-2-SOLAR-10.7B',
 'https://huggingface.co/meta-llama/Llama-2-7b-chat-hf',
 'https://huggingface.co/WizardLM/WizardCoder-33B-V1.1',
 'https://huggingface.co/upstage/SOLAR-10.7B-Instruct-v1.0',
 'https://huggingface.co/cognitivecomputations/dolphin-2.5-mixtral-8x7b',
 'https://huggingface.co/jondurbin/bagel-dpo-34b-v0.2',
 'https://huggingface.co/meta-llama/Llama-2-7b',
 'https://huggingface.co/cognitivecomputations/dolphin-2.7-mixtral-8x7b',
 'https://huggingface.co/eastwind/tinymix-8x1b-chat',
 'https://huggingface.co/intfloat/multilingual-e5-large',
 'https://huggingface.co/cognitivecomputations/dolphin-2.6-mistral-7b-

In [24]:
from src.io import save_json

save_json(yes_models, os.path.join(data_root, "ft_models.json"))

In [20]:
failed_cards = response[1]
print(f"Failed to answer for these models:\n{failed_cards}")

Failed to answer for these models:
[]
